Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [122]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [123]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [124]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [125]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [126]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss += 1e-3 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [127]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.528114
Minibatch accuracy: 10.9%
Validation accuracy: 13.3%
Minibatch loss at step 500: 2.644883
Minibatch accuracy: 80.5%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.879375
Minibatch accuracy: 79.7%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.967859
Minibatch accuracy: 85.9%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.850982
Minibatch accuracy: 86.7%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 0.809842
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.756963
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.8%


In [128]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_neurons = 1024
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_neurons]))
  biases = tf.Variable(tf.zeros([num_neurons]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_neurons, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2)
  loss += 1e-3 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [129]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 668.681763
Minibatch accuracy: 10.2%
Validation accuracy: 29.1%
Minibatch loss at step 500: 196.446945
Minibatch accuracy: 84.4%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 115.432030
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 68.577209
Minibatch accuracy: 89.1%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 41.177334
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 25.151529
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.512388
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [130]:
'''

num_steps = 3001

train_dataset = train_dataset[:(100 * batch_size), :]
train_labels = train_labels[:(100 * batch_size), :]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
'''

'\n\nnum_steps = 3001\n\ntrain_dataset = train_dataset[:(100 * batch_size), :]\ntrain_labels = train_labels[:(100 * batch_size), :]\n\nwith tf.Session(graph=graph) as session:\n  tf.initialize_all_variables().run()\n  print("Initialized")\n  for step in range(num_steps):\n    # Pick an offset within the training data, which has been randomized.\n    # Note: we could use better randomization across epochs.\n    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)\n    # Generate a minibatch.\n    batch_data = train_dataset[offset:(offset + batch_size), :]\n    batch_labels = train_labels[offset:(offset + batch_size), :]\n    # Prepare a dictionary telling the session where to feed the minibatch.\n    # The key of the dictionary is the placeholder node of the graph to be fed,\n    # and the value is the numpy array to feed to it.\n    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}\n    _, l, predictions = session.run(\n      [optimizer, loss, t

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [131]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_neurons = 1024
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_neurons]))
  biases = tf.Variable(tf.zeros([num_neurons]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_neurons, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  dropout = 0.9

  def model(dataset, keep_prob) :
    h1 = tf.matmul(tf.nn.dropout(dataset, keep_prob), weights) + biases
    h2 = tf.nn.dropout(tf.nn.relu(h1), keep_prob)
    h3 = tf.matmul(h2, weights2) + biases2
    return h3
    
  # Training computation.
  logits = model(tf_train_dataset, keep_prob = dropout) #tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2)
  loss += 1e-3 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, keep_prob = 1.0))
    #tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  test_prediction = tf.nn.softmax(model(tf_test_dataset, keep_prob = 1.0))
    #tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [132]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 672.931519
Minibatch accuracy: 4.7%
Validation accuracy: 27.6%
Minibatch loss at step 500: 201.231659
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 117.932793
Minibatch accuracy: 75.8%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.501724
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 40.828979
Minibatch accuracy: 88.3%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 24.802879
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 15.261041
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Test accuracy: 92.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [208]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_neurons = 1024
  num_neurons2 = 256
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_neurons], stddev = 0.1))
  biases = tf.Variable(tf.zeros([num_neurons]))

  weights2 = tf.Variable(
     tf.truncated_normal([num_neurons, num_neurons2], stddev = 0.1))
  biases2 = tf.Variable(tf.zeros([num_neurons2]))   
  
  weights3 = tf.Variable(
    tf.truncated_normal([num_neurons2, num_labels], stddev = 0.1))
  biases3 = tf.Variable(tf.zeros([num_labels]))

  dropout = 1.0

  def model(dataset, keep_prob) :
    dataset = tf.nn.dropout(dataset, keep_prob)
    h1 = tf.matmul(dataset, weights) + biases
    h1 = tf.nn.dropout(tf.nn.relu(h1), keep_prob)
    h2 = tf.matmul(h1, weights2) + biases2
    h2 = tf.nn.dropout(tf.nn.relu(h2), keep_prob)
    h3 = tf.matmul(h2, weights3) + biases3
    return h3
    
  # Training computation.
  logits = model(tf_train_dataset, keep_prob = dropout)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))/3
  loss += 5e-4 * regularizers
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, keep_prob = 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, keep_prob = 1.0))
  print('model prepared')

model prepared


In [209]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.819955
Minibatch accuracy: 12.5%
Validation accuracy: 13.3%
Minibatch loss at step 500: 1.063284
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 1.128452
Minibatch accuracy: 84.4%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 0.846539
Minibatch accuracy: 92.2%
Validation accuracy: 87.3%
Minibatch loss at step 2000: 0.825616
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 2500: 0.795657
Minibatch accuracy: 89.8%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 0.833171
Minibatch accuracy: 89.1%
Validation accuracy: 88.1%
Minibatch loss at step 3500: 0.799589
Minibatch accuracy: 89.1%
Validation accuracy: 88.6%
Minibatch loss at step 4000: 0.676355
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 4500: 0.640314
Minibatch accuracy: 90.6%
Validation accuracy: 88.3%
Minibatch loss at step 5000: 0.675539
Minibatch accuracy: 92.2%
Validation accurac